# Расширенное тестирование модуля формирования портфеля

Этот ноутбук демонстрирует расширенный цикл работы с модулем tvr_service.pipeline.portfolio — от загрузки данных по инструментам до расчёта распределения капитала с использованием индикаторного инструмента и параметров.


## 1. Подготовка окружения

Импортируем необходимые библиотеки и подключим модуль портфеля. Добавляем каталог src в sys.path, чтобы можно было использовать код проекта без установки пакета.


In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
SRC_DIR = PROJECT_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

from tvr_service.pipeline import (
    allocations_to_frame,
    build_portfolio,
    filter_by_suffix,
    filter_by_whitelist,
    load_securities,
    load_whitelist,
)

import pandas as pd


## 2. Настройка параметров теста

Задаём базовые параметры: объём капитала, необходимость скачивания свежих данных с MOEX, путь к локальному CSV (если он есть), фильтрацию по суффиксу тикера, количество параметров и индикаторный инструмент.


In [2]:
CAPITAL = 3_000_000  # общий капитал, руб.
PREFER_REMOTE = True   # при True сначала пробуем скачать данные с MOEX
SEC_DATA_FILE = None   # можно указать путь к локальному sec_tvr.csv
SUFFIX_FILTER = 'Z5'   # например, 'H5', 'M5' и т.п. для фьючерсов
NUM_PARAMS = 3         # количество параметров для каждого инструмента в портфеле
SEC_IND = 'MXZ5'       # код индикаторного инструмента (пример)

# Параметры для расчета float значений
float_1_x = 0.033  # коэффициент для float_1
float_2_x = 0.01   # коэффициент для float_2
float_3_x = 0.01   # значение для float_3
float_4_x = 0.05   # значение для float_4


## 3. Загрузка справочника инструментов

Попробуем получить таблицу инструментов. Функция load_securities автоматически рассчитает поле full_price и добавит тикеры TI* (клонов TB*). В процессе будет использован список из интернета или локального файла.


In [3]:
securities = load_securities(sec_data_file=SEC_DATA_FILE, prefer_remote=PREFER_REMOTE)
print(f"Загружено инструментов: {len(securities)}")
securities.head()


Загружено инструментов: 400


,SECID,MINSTEP,STEPPRICE,PREVSETTLEPRICE,INITIALMARGIN,BUYSELLFEE,SCALPERFEE,SHORTNAME,full_price,CODE,PRICE,SELLDEPO,base_code
0,AEH6,0.001,1.0,24.550,3834.96,1.13,0.57,AED-3.26,24550.0,AEH6,24.550,3834.96,AED
1,AEM6,0.001,1.0,25.120,4016.06,1.16,0.58,AED-6.26,25120.0,AEM6,25.120,4016.06,AED
2,AEZ5,0.001,1.0,23.478,3736.88,1.08,0.54,AED-12.25,23478.0,AEZ5,23.478,3736.88,AED
3,AFH6,1.000,1.0,6020.000,1628.23,1.20,0.60,AFLT-3.26,6020.0,AFH6,6020.000,1628.23,AFLT
4,AFZ5,1.000,1.0,5793.000,1538.73,1.15,0.58,AFLT-12.25,5793.0,AFZ5,5793.000,1538.73,AFLT


## 4. Получение цены индикаторного инструмента

Найдем цену индикаторного инструмента в справочнике и сохраним её в переменную SEC_IND_PRICE.


In [4]:
# Поиск индикаторного инструмента в справочнике
indicator_security = securities[securities['SECID'] == SEC_IND]

if not indicator_security.empty:
    SEC_IND_PRICE = indicator_security['full_price'].iloc[0]
    print(f"Индикаторный инструмент {SEC_IND} найден. Цена: {SEC_IND_PRICE:,.2f} руб.")
    print(f"Краткое название: {indicator_security['SHORTNAME'].iloc[0]}")
else:
    print(f"Индикаторный инструмент {SEC_IND} не найден в справочнике!")
    print("Доступные инструменты с похожими кодами:")
    similar_securities = securities[securities['SECID'].str.contains(SEC_IND[:3], case=False)]
    if not similar_securities.empty:
        print(similar_securities[['SECID', 'SHORTNAME']].head())
    else:
        print("Похожих инструментов не найдено.")
    SEC_IND_PRICE = None


Индикаторный инструмент MXZ5 найден. Цена: 281,500.00 руб.
Краткое название: MIX-12.25


## 5. Фильтрация справочника

Сначала ограничиваем список инструментов по выбранному `SUFFIX_FILTER`, затем применяем whitelist. Так можно сузить вселенную до нужной серии и далее оставить только интересующие базовые активы.


In [5]:
try:
    whitelist = load_whitelist()
    print(f'Инструментов в whitelist: {len(whitelist)}')
except FileNotFoundError:
    whitelist = None
    print('Файл whitelist не найден. Будем использовать полную вселенную инструментов.')

# Фильтрация по суффиксу
base_selection = filter_by_suffix(securities, SUFFIX_FILTER)
print(f'Инструментов после фильтра по суффиксу {SUFFIX_FILTER}: {len(base_selection)}')

# Фильтрация по whitelist
if whitelist and not base_selection.empty:
    filtered_securities = filter_by_whitelist(base_selection, whitelist)
    print(f'Инструментов после применения whitelist: {len(filtered_securities)}')
elif whitelist:
    filtered_securities = pd.DataFrame(columns=base_selection.columns if not base_selection.empty else securities.columns)
    print('После фильтра по суффиксу подходящих инструментов нет.')
else:
    filtered_securities = base_selection

if not filtered_securities.empty:
    print('Примеры отфильтрованных инструментов:')
    display(filtered_securities[['SECID', 'SHORTNAME', 'base_code', 'full_price']].head())
    
    # Проверка уникальности по base_code
    print(f"\nПроверка уникальности по base_code:")
    unique_base_codes = filtered_securities['base_code'].nunique()
    total_instruments = len(filtered_securities)
    print(f"Уникальных base_code: {unique_base_codes}")
    print(f"Общее количество инструментов: {total_instruments}")
    
    if unique_base_codes != total_instruments:
        print("⚠️  ВНИМАНИЕ: Найдены дубликаты по base_code!")
        duplicates = filtered_securities.groupby('base_code').size()
        duplicates = duplicates[duplicates > 1]
        print(f"Дублирующиеся base_code ({len(duplicates)}):")
        for base_code, count in duplicates.head(10).items():
            print(f"  {base_code}: {count} инструментов")
            # Показываем примеры дубликатов
            examples = filtered_securities[filtered_securities['base_code'] == base_code][['SECID', 'SHORTNAME', 'base_code']]
            print(f"    Примеры: {examples['SECID'].tolist()}")
    else:
        print("✅ Все base_code уникальны")
        
else:
    print('После фильтрации инструментов не найдено.')


Инструментов в whitelist: 44
Инструментов после фильтра по суффиксу Z5: 131
Инструментов после применения whitelist: 45
Примеры отфильтрованных инструментов:


,SECID,SHORTNAME,base_code,full_price
0,AFZ5,AFLT-12.25,AFLT,5793.0
1,AKZ5,AFKS-12.25,AFKS,14687.0
2,ALZ5,ALRS-12.25,ALRS,4627.0
3,BNZ5,BANE-12.25,BANE,1600.0
4,CMZ5,CBOM-12.25,CBOM,7539.0



Проверка уникальности по base_code:
Уникальных base_code: 44
Общее количество инструментов: 45
⚠️  ВНИМАНИЕ: Найдены дубликаты по base_code!
Дублирующиеся base_code (1):
  T: 2 инструментов
    Примеры: ['TBZ5', 'TIZ5']


## 5.2. Альтернативный диалог (без ipywidgets)

Если ipywidgets недоступны, используем простой текстовый интерфейс для выбора дубликатов.

In [6]:
def remove_duplicates_simple(df, base_code_column='base_code'):
    """
    Простая функция для удаления дублирующихся инструментов по base_code.
    Использует текстовый интерфейс вместо ipywidgets.
    
    Args:
        df: DataFrame с инструментами
        base_code_column: название колонки для проверки дубликатов
    
    Returns:
        DataFrame без дубликатов
    """
    import pandas as pd
    
    # Находим дубликаты
    duplicates = df.groupby(base_code_column).size()
    duplicates = duplicates[duplicates > 1]
    
    if len(duplicates) == 0:
        print("✅ Дубликатов не найдено!")
        return df
    
    print(f"🔍 Найдено {len(duplicates)} групп дубликатов:")
    
    indices_to_remove = []
    
    for base_code in duplicates.index:
        group_data = df[df[base_code_column] == base_code]
        print(f"\n📋 Группа '{base_code}' ({len(group_data)} инструментов):")
        
        # Показываем инструменты в группе
        for i, (idx, row) in enumerate(group_data.iterrows()):
            print(f"  {i+1}. {row['SECID']} - {row['SHORTNAME']} (цена: {row['full_price']})")
        
        # Запрашиваем выбор
        print(f"\n❓ Какие инструменты из группы '{base_code}' удалить?")
        print("   Введите номера через запятую (например: 1,3) или 'all' для удаления всех кроме первого:")
        
        try:
            choice = input("Ваш выбор: ").strip()
            
            if choice.lower() == 'all':
                # Удаляем все кроме первого
                indices_to_remove.extend(group_data.index[1:].tolist())
                print(f"   ✅ Будет удалено {len(group_data)-1} инструментов (останется первый)")
            elif choice:
                # Парсим номера
                numbers = [int(x.strip()) for x in choice.split(',') if x.strip().isdigit()]
                if numbers:
                    # Проверяем валидность номеров
                    valid_numbers = [n for n in numbers if 1 <= n <= len(group_data)]
                    if valid_numbers:
                        # Получаем индексы для удаления
                        indices_to_remove.extend([group_data.index[n-1] for n in valid_numbers])
                        print(f"   ✅ Будет удалено {len(valid_numbers)} инструментов")
                    else:
                        print("   ⚠️ Некорректные номера, пропускаем группу")
                else:
                    print("   ⚠️ Некорректный ввод, пропускаем группу")
            else:
                print("   ⏭️ Группа пропущена")
                
        except (ValueError, KeyboardInterrupt):
            print("   ⚠️ Некорректный ввод или прерывание, пропускаем группу")
            continue
    
    if indices_to_remove:
        print(f"\n🗑️ Удаляем {len(indices_to_remove)} выбранных инструментов...")
        
        # Удаляем выбранные инструменты
        df_cleaned = df.drop(indices_to_remove).reset_index(drop=True)
        
        print(f"✅ Очистка завершена!")
        print(f"   Исходное количество: {len(df)}")
        print(f"   После очистки: {len(df_cleaned)}")
        print(f"   Удалено: {len(df) - len(df_cleaned)}")
        
        # Проверяем результат
        remaining_duplicates = df_cleaned.groupby(base_code_column).size()
        remaining_duplicates = remaining_duplicates[remaining_duplicates > 1]
        
        if len(remaining_duplicates) == 0:
            print("✅ Все дубликаты успешно удалены!")
        else:
            print(f"⚠️ Остались дубликаты: {len(remaining_duplicates)} групп")
        
        return df_cleaned
    else:
        print("ℹ️ Ничего не выбрано для удаления.")
        return df

# Попробуем использовать простую версию, если интерактивная не работает
try:
    if 'filtered_securities' in locals() and not filtered_securities.empty:
        print("🔧 Запускаем простой диалог удаления дубликатов...")
        filtered_securities_cleaned = remove_duplicates_simple(filtered_securities)
        
        # Обновляем переменную
        filtered_securities = filtered_securities_cleaned
        print(f"\n📊 Итоговое количество инструментов: {len(filtered_securities)}")
    else:
        print("❌ Нет данных для обработки дубликатов")
except Exception as e:
    print(f"❌ Ошибка при обработке дубликатов: {e}")
    print("Продолжаем с исходными данными...")


🔧 Запускаем простой диалог удаления дубликатов...
🔍 Найдено 1 групп дубликатов:

📋 Группа 'T' (2 инструментов):
  1. TBZ5 - T-12.25 (цена: 3222.0)
  2. TIZ5 - T-12.25 (цена: 3222.0)

❓ Какие инструменты из группы 'T' удалить?
   Введите номера через запятую (например: 1,3) или 'all' для удаления всех кроме первого:
   ✅ Будет удалено 1 инструментов

🗑️ Удаляем 1 выбранных инструментов...
✅ Очистка завершена!
   Исходное количество: 45
   После очистки: 44
   Удалено: 1
✅ Все дубликаты успешно удалены!

📊 Итоговое количество инструментов: 44


## 6. Создание основы для построения портфеля

Создаем основу для портфеля с расчетом весовых коэффициентов относительно индикаторного инструмента.


In [7]:
if SEC_IND_PRICE is not None and not filtered_securities.empty:
    print(f"Создаем основу портфеля с {NUM_PARAMS} параметрами для каждого инструмента")
    print(f"Индикаторный инструмент: {SEC_IND} (цена: {SEC_IND_PRICE:,.2f} руб.)")
    
    # Создаем список для хранения данных портфеля
    portfolio_basis = []
    
    for idx, row in filtered_securities.iterrows():
        sec_0 = row['SECID']
        sec_0_price = row['full_price']
        sec_0_GO = row['INITIALMARGIN']
        sec_1 = SEC_IND
        
        # Расчет весового коэффициента W_0
        W_0 = (sec_0_price / SEC_IND_PRICE * 1000000) // 1000 / 1000
        
        # Расчет float значений
        float_1 = max(1, int(sec_0_price / 100 * float_1_x))
        float_2 = max(1, int(sec_0_price / 100 * float_2_x))
        float_3 = float_3_x
        float_4 = float_4_x
        
        # Формирование колонки SM
        SM = f"{float_1};{float_2};{float_3};{float_4}"
        
        portfolio_basis.append({
            'Sec_0': sec_0,
            'sec_0_price': sec_0_price,
            'sec_0_GO': sec_0_GO,
            'sec_1': sec_1,
            'W_0': W_0,
            'SM': SM
        })
    
    # Преобразуем в DataFrame
    portfolio_basis_df = pd.DataFrame(portfolio_basis)
    
    print(f"\nСоздана основа портфеля для {len(portfolio_basis_df)} инструментов")
    print("\nПримеры записей:")
    # display(portfolio_basis_df.head())
    
    # Статистика по весовым коэффициентам
    print(f"\nСтатистика весовых коэффициентов W_0:")
    print(f"Минимум: {portfolio_basis_df['W_0'].min():.6f}")
    print(f"Максимум: {portfolio_basis_df['W_0'].max():.6f}")
    print(f"Среднее: {portfolio_basis_df['W_0'].mean():.6f}")
    print(f"Медиана: {portfolio_basis_df['W_0'].median():.6f}")
    
else:
    print("Ошибка: Не удалось создать основу портфеля.")
    if SEC_IND_PRICE is None:
        print("- Индикаторный инструмент не найден")
    if filtered_securities.empty:
        print("- Нет отфильтрованных инструментов")


Создаем основу портфеля с 3 параметрами для каждого инструмента
Индикаторный инструмент: MXZ5 (цена: 281,500.00 руб.)

Создана основа портфеля для 44 инструментов

Примеры записей:

Статистика весовых коэффициентов W_0:
Минимум: 0.002000
Максимум: 0.402000
Среднее: 0.054682
Медиана: 0.023000


## 7. Размножение строк по NUM_PARAMS

Создаем strategy_id для каждой комбинации инструмента и параметра, размножая строки по количеству параметров.


In [8]:
if 'portfolio_basis_df' in locals() and not portfolio_basis_df.empty:
    print("Размножаем строки по NUM_PARAMS...")
    
    # Создаем список параметров от 1 до NUM_PARAMS
    params_list = list(range(1, NUM_PARAMS + 1))
    print(f"Список параметров: {params_list}")
    
    # Создаем размноженные строки
    expanded_portfolio = []
    
    for idx, row in portfolio_basis_df.iterrows():
        for param_num in params_list:
            # Создаем strategy_id
            strategy_id = f"{row['Sec_0']}_{param_num}"
            
            # Копируем все данные строки
            expanded_row = row.copy()
            expanded_row['strategy_id'] = strategy_id
            expanded_row['param_num'] = param_num
            
            # Переупорядочиваем колонки, чтобы strategy_id была первой
            expanded_row = expanded_row.reindex(['strategy_id'] + [col for col in expanded_row.index if col != 'strategy_id'])
            
            expanded_portfolio.append(expanded_row)
    
    # Преобразуем в DataFrame
    expanded_portfolio_df = pd.DataFrame(expanded_portfolio)
    
    print(f"\nРазмножение завершено:")
    print(f"Исходных инструментов: {len(portfolio_basis_df)}")
    print(f"Параметров на инструмент: {NUM_PARAMS}")
    print(f"Итого строк: {len(expanded_portfolio_df)}")
    print(f"Ожидалось: {len(portfolio_basis_df) * NUM_PARAMS}")
    
    print("\nПримеры strategy_id:")
    sample_strategies = expanded_portfolio_df['strategy_id'].head(10).tolist()
    print(sample_strategies)
    
    print("\nПримеры записей:")
    display(expanded_portfolio_df[['strategy_id', 'Sec_0', 'param_num', 'W_0', 'SM']].head())
    
    # Проверяем уникальность strategy_id
    unique_strategies = expanded_portfolio_df['strategy_id'].nunique()
    total_strategies = len(expanded_portfolio_df)
    print(f"\nПроверка уникальности:")
    print(f"Уникальных strategy_id: {unique_strategies}")
    print(f"Общее количество строк: {total_strategies}")
    print(f"Совпадают: {unique_strategies == total_strategies}")
    
else:
    print("Ошибка: Не удалось размножить строки.")
    print("Убедитесь, что основа портфеля была создана успешно.")


Размножаем строки по NUM_PARAMS...
Список параметров: [1, 2, 3]

Размножение завершено:
Исходных инструментов: 44
Параметров на инструмент: 3
Итого строк: 132
Ожидалось: 132

Примеры strategy_id:
['AFZ5_1', 'AFZ5_2', 'AFZ5_3', 'AKZ5_1', 'AKZ5_2', 'AKZ5_3', 'ALZ5_1', 'ALZ5_2', 'ALZ5_3', 'BNZ5_1']

Примеры записей:


,strategy_id,Sec_0,param_num,W_0,SM
0,AFZ5_1,AFZ5,1,0.020,1;1;0.01;0.05
0,AFZ5_2,AFZ5,2,0.020,1;1;0.01;0.05
0,AFZ5_3,AFZ5,3,0.020,1;1;0.01;0.05
1,AKZ5_1,AKZ5,1,0.052,4;1;0.01;0.05
1,AKZ5_2,AKZ5,2,0.052,4;1;0.01;0.05



Проверка уникальности:
Уникальных strategy_id: 132
Общее количество строк: 132
Совпадают: True


## 8.2. Тестирование жадного алгоритма

Давайте протестируем логику распределения на конкретном примере и посмотрим, как работает алгоритм.


In [9]:
final_portfolio = expanded_portfolio_df.copy()
from tvr_service.pipeline import build_portfolio_advanced
# Тестируем функцию на реальных данных AFZ5
print("=== ТЕСТ НА РЕАЛЬНЫХ ДАННЫХ AFZ5 ===")

if 'final_portfolio' in locals() and not final_portfolio.empty:
    # Берем только AFZ5 из реальных данных
    afz5_real = final_portfolio[final_portfolio['Sec_0'] == 'AFZ5'].copy()
    
    if not afz5_real.empty:
        print(f"Найдено {len(afz5_real)} строк AFZ5")
        print("Данные:")
        print(afz5_real[['Sec_0', 'param_num', 'sec_0_price', 'strategy_id']].to_string(index=False))
        print()
        
        # Применяем функцию только к AFZ5
        capital_per_sec = CAPITAL / final_portfolio['Sec_0'].nunique()
        print(f"Капитал на инструмент: {capital_per_sec:,.0f} руб.")
        print(f"Цена AFZ5: {afz5_real['sec_0_price'].iloc[0]:,.0f} руб.")
        print(f"Максимум лотов: {int(capital_per_sec // afz5_real['sec_0_price'].iloc[0])}")
        print()
        
        # Тестируем функцию
        result_afz5 = build_portfolio_advanced(
            data=afz5_real,
            capital=capital_per_sec,
            security_column="Sec_0",
            price_column="sec_0_price",
            param_num_column="param_num"
        )
        
        print("Результат для AFZ5:")
        for _, row in result_afz5.iterrows():
            print(f"  {row['strategy_id']}: {row['estimated_lots']} лотов, {row['used_capital']:,.0f} руб.")
        
        total_lots = result_afz5['estimated_lots'].sum()
        total_used = result_afz5['used_capital'].sum()
        expected_lots = int(capital_per_sec // afz5_real['sec_0_price'].iloc[0])
        
        print(f"\nИтого: {total_lots} лотов, {total_used:,.0f} руб.")
        print(f"Ожидалось: {expected_lots} лотов")
        print(f"Совпадает: {'ДА' if total_lots == expected_lots else 'НЕТ'}")
        
        if total_lots != expected_lots:
            print(f"\n❌ ПРОБЛЕМА: Получили {total_lots} лотов, а должно быть {expected_lots}")
            print("Возможные причины:")
            print("1. Разные цены в строках AFZ5")
            print("2. Проблема с param_num")
            print("3. Ошибка в алгоритме функции")
        else:
            print(f"\n✅ ОТЛИЧНО: Функция работает правильно на реальных данных!")
            
    else:
        print("❌ AFZ5 не найден в final_portfolio")
        
else:
    print("❌ final_portfolio не доступен")


=== ТЕСТ НА РЕАЛЬНЫХ ДАННЫХ AFZ5 ===
Найдено 3 строк AFZ5
Данные:
Sec_0  param_num  sec_0_price strategy_id
 AFZ5          1       5793.0      AFZ5_1
 AFZ5          2       5793.0      AFZ5_2
 AFZ5          3       5793.0      AFZ5_3

Капитал на инструмент: 68,182 руб.
Цена AFZ5: 5,793 руб.
Максимум лотов: 11

Результат для AFZ5:
  AFZ5_1: 3 лотов, 17,379 руб.
  AFZ5_2: 3 лотов, 17,379 руб.
  AFZ5_3: 3 лотов, 17,379 руб.

Итого: 9 лотов, 52,137 руб.
Ожидалось: 11 лотов
Совпадает: НЕТ

❌ ПРОБЛЕМА: Получили 9 лотов, а должно быть 11
Возможные причины:
1. Разные цены в строках AFZ5
2. Проблема с param_num
3. Ошибка в алгоритме функции


## 8. Формирование финального каркаса портфеля

Создаем финальную структуру портфеля с анализом результатов.


## 8.1. Тестирование новой функции build_portfolio_advanced

Теперь протестируем новую функцию `build_portfolio_advanced`, которая может работать с любым DataFrame и правильно распределять капитал по группам инструментов и параметрам.


In [ ]:
final_portfolio = expanded_portfolio_df.copy()
# Импортируем новую функцию
from tvr_service.pipeline import build_portfolio_advanced

if 'final_portfolio' in locals() and not final_portfolio.empty:
    print("Тестируем новую функцию build_portfolio_advanced...")
    print(f"Исходные данные: {len(final_portfolio)} строк, {final_portfolio['Sec_0'].nunique()} уникальных инструментов")
    
    # Применяем новую функцию для распределения капитала
    portfolio_with_allocation = build_portfolio_advanced(
        data=final_portfolio,
        capital=CAPITAL,
        security_column="Sec_0",
        price_column="sec_0_price",
        param_num_column="param_num"
    )
    
    print(f"Результат: {len(portfolio_with_allocation)} строк")
    print("\nПример результата:")
    display(portfolio_with_allocation[['strategy_id', 'Sec_0', 'param_num', 'sec_0_price', 'allocation', 'estimated_lots', 'used_capital', 'unused_capital']].head(10))
    
elif 'expanded_portfolio_df' in locals() and not expanded_portfolio_df.empty:
    print("⚠️  final_portfolio не найден, но expanded_portfolio_df доступен.")
    print("Создаем final_portfolio из expanded_portfolio_df...")
    
    # Создаем final_portfolio из expanded_portfolio_df
    final_portfolio = expanded_portfolio_df.copy()
    print(f"Создан final_portfolio: {len(final_portfolio)} строк")
    
    # Теперь применяем новую функцию
    portfolio_with_allocation = build_portfolio_advanced(
        data=final_portfolio,
        capital=CAPITAL,
        security_column="Sec_0",
        price_column="sec_0_price",
        param_num_column="param_num"
    )
    
    print(f"Результат: {len(portfolio_with_allocation)} строк")
    print("\nПример результата:")
    display(portfolio_with_allocation[['strategy_id', 'Sec_0', 'param_num', 'sec_0_price', 'allocation', 'estimated_lots', 'used_capital', 'unused_capital']].head(10))
    
else:
    print("❌ Ошибка: Не найдены необходимые данные.")
    print("Доступные переменные:")
    available_vars = [var for var in locals() if 'portfolio' in var.lower() or 'expanded' in var.lower()]
    if available_vars:
        print(f"  - {', '.join(available_vars)}")
    else:
        print("  - Нет переменных с 'portfolio' или 'expanded' в названии")
    print("\nУбедитесь, что выполнили все предыдущие шаги блокнота.")


In [ ]:
final_portfolio.shape

In [ ]:
portfolio_with_allocation

In [ ]:
# Анализ результатов распределения капитала
if 'portfolio_with_allocation' in locals():
    print("=== АНАЛИЗ РАСПРЕДЕЛЕНИЯ КАПИТАЛА ===\n")
    
    # Общая статистика
    total_used = portfolio_with_allocation['used_capital'].sum()
    total_unused = portfolio_with_allocation['unused_capital'].sum()
    utilization_rate = total_used / CAPITAL * 100
    
    print(f"Общий капитал: {CAPITAL:,.0f} руб.")
    print(f"Использовано: {total_used:,.0f} руб. ({utilization_rate:.1f}%)")
    print(f"Неиспользовано: {total_unused:,.0f} руб. ({100-utilization_rate:.1f}%)")
    
    # Статистика по инструментам
    print(f"\n=== СТАТИСТИКА ПО ИНСТРУМЕНТАМ ===")
    instrument_stats = portfolio_with_allocation.groupby('Sec_0').agg({
        'allocation': 'first',  # allocation одинаковая для всех строк инструмента
        'estimated_lots': 'sum',
        'used_capital': 'sum',
        'unused_capital': 'sum',
        'param_num': 'count'
    }).rename(columns={'param_num': 'param_count'})
    
    print(f"Уникальных инструментов: {len(instrument_stats)}")
    print(f"Капитал на инструмент: {CAPITAL / len(instrument_stats):,.0f} руб.")
    print("\nТоп-5 инструментов по используемому капиталу:")
    top_instruments = instrument_stats.nlargest(5, 'used_capital')[['used_capital', 'estimated_lots', 'param_count']]
    display(top_instruments)
    
    # Пример распределения внутри одного инструмента
    print(f"\n=== ПРИМЕР РАСПРЕДЕЛЕНИЯ ВНУТРИ ИНСТРУМЕНТА ===")
    sample_instrument = portfolio_with_allocation['Sec_0'].iloc[0]
    sample_group = portfolio_with_allocation[portfolio_with_allocation['Sec_0'] == sample_instrument]
    print(f"Инструмент: {sample_instrument}")
    print(f"Цена: {sample_group['sec_0_price'].iloc[0]:,.0f} руб.")
    print(f"Параметров: {len(sample_group)}")
    
    sample_display = sample_group[['strategy_id', 'param_num', 'allocation', 'estimated_lots', 'used_capital', 'unused_capital']].sort_values('param_num')
    display(sample_display)
    
    print(f"\nПроверка суммы лотов: {sample_group['estimated_lots'].sum()} лотов")
    expected_lots = int((CAPITAL / len(instrument_stats)) // sample_group['sec_0_price'].iloc[0])
    print(f"Ожидаемая сумма лотов: {expected_lots} лотов")
    print(f"Совпадает: {'✅' if sample_group['estimated_lots'].sum() == expected_lots else '❌'}")
    
else:
    print("Переменная portfolio_with_allocation не найдена.")


In [ ]:
# Демонстрация гибкости функции: тестируем с другим столбцом для расчетов
if 'portfolio_with_allocation' in locals():
    print("=== ТЕСТИРОВАНИЕ С ДРУГИМ СТОЛБЦОМ ДЛЯ РАСЧЕТОВ ===\n")
    print("Сравниваем результаты при использовании 'sec_0_price' и 'sec_0_GO' для расчетов:\n")
    
    # Тестируем с sec_0_GO (гарантийное обеспечение)
    portfolio_with_go = build_portfolio_advanced(
        data=final_portfolio,
        capital=CAPITAL,
        security_column="Sec_0",
        price_column="sec_0_GO",  # Используем гарантийное обеспечение
        param_num_column="param_num"
    )
    
    # Сравнение для одного инструмента
    sample_sec = final_portfolio['Sec_0'].iloc[0]
    
    print(f"Пример для инструмента: {sample_sec}")
    
    # Данные по цене
    price_group = portfolio_with_allocation[portfolio_with_allocation['Sec_0'] == sample_sec]
    price_summary = {
        'Основа расчета': f"sec_0_price ({price_group['sec_0_price'].iloc[0]:,.0f} руб.)",
        'Общие лоты': price_group['estimated_lots'].sum(),
        'Использованный капитал': price_group['used_capital'].sum(),
        'Неиспользованный капитал': price_group['unused_capital'].sum()
    }
    
    # Данные по ГО
    go_group = portfolio_with_go[portfolio_with_go['Sec_0'] == sample_sec]
    go_summary = {
        'Основа расчета': f"sec_0_GO ({go_group['sec_0_GO'].iloc[0]:,.0f} руб.)",
        'Общие лоты': go_group['estimated_lots'].sum(),
        'Использованный капитал': go_group['used_capital'].sum(),
        'Неиспользованный капитал': go_group['unused_capital'].sum()
    }
    
    comparison_df = pd.DataFrame([price_summary, go_summary], index=['По цене', 'По ГО'])
    display(comparison_df)
    
    print(f"\nВывод: Функция работает с любым столбцом для расчетов!")
    print(f"При использовании ГО получаем больше лотов, так как ГО обычно меньше полной цены контракта.")
    
else:
    print("Ошибка: portfolio_with_allocation не создан.")


In [22]:
test_portfolio = final_portfolio.iloc[0:3]
test_portfolio

,strategy_id,Sec_0,sec_0_price,sec_0_GO,sec_1,W_0,SM,param_num
0,AFZ5_1,AFZ5,5793.0,1538.73,MXZ5,0.02,1;1;0.01;0.05,1
0,AFZ5_2,AFZ5,5793.0,1538.73,MXZ5,0.02,1;1;0.01;0.05,2
0,AFZ5_3,AFZ5,5793.0,1538.73,MXZ5,0.02,1;1;0.01;0.05,3


In [19]:
# def build_portfolio_advanced(
#     data: test_portfolio,
#     capital: 68300,
#     security_column: str = "Sec_0",
#     price_column: str = "sec_0_price",
#     param_num_column: str = "param_num",
# ) -> pd.DataFrame:
#     """
#     Построение портфеля с распределением капитала по группам инструментов и параметрам.
    
#     Args:
#         data: DataFrame с данными об инструментах и параметрах
#         capital: Общий капитал для распределения
#         security_column: Название столбца с инструментами для группировки
#         price_column: Название столбца с ценами для расчета лотов
#         param_num_column: Название столбца с номерами параметров в группе
    
#     Returns:
#         DataFrame с добавленными столбцами: allocation, estimated_lots, used_capital, unused_capital
#     """
#     if capital <= 0:
#         raise ValueError("Капитал должен быть положительным числом")
    
#     if data.empty:
#         raise ValueError("DataFrame не может быть пустым")
    
#     # Проверяем наличие необходимых столбцов
#     required_columns = [security_column, price_column, param_num_column]
#     missing_columns = [col for col in required_columns if col not in data.columns]
#     if missing_columns:
#         raise ValueError(f"Отсутствуют столбцы: {missing_columns}")
    
#     # Создаем копию данных для работы
#     result_df = data.copy()
    
#     # Инициализируем новые столбцы
#     result_df["allocation"] = 0.0
#     result_df["estimated_lots"] = 0
#     result_df["used_capital"] = 0.0
#     result_df["unused_capital"] = 0.0
    
#     # Получаем количество уникальных инструментов
#     unique_securities = result_df[security_column].nunique()
#     capital_per_security = capital / unique_securities
    
#     # Группируем по инструментам
#     grouped = result_df.groupby(security_column)
    
#     for security_name, group in grouped:
#         # Сортируем группу по номеру параметра
#         group_sorted = group.sort_values(param_num_column)
        
#         # Получаем цену инструмента (берем из первой строки)
#         instrument_price = float(group_sorted[price_column].iloc[0])
        
#         if instrument_price <= 0:
#             # Если цена 0 или отрицательная, пропускаем инструмент
#             continue
        
#         # Количество строк в группе
#         param_count = len(group_sorted)

#         if param_count == 0:
#             continue

#         # ЖАДНЫЙ АЛГОРИТМ: выдаем лоты, работая с общим бюджетом инструмента
#         remaining_capital = capital_per_security
#         lots_per_row = [0] * param_count  # Количество лотов для каждой строки

#         while remaining_capital >= instrument_price:
#             min_lots_idx = lots_per_row.index(min(lots_per_row))
#             lots_per_row[min_lots_idx] += 1
#             remaining_capital -= instrument_price

#         used_per_row = [lots * instrument_price for lots in lots_per_row]
#         total_used = sum(used_per_row)
#         leftover_capital = max(capital_per_security - total_used, 0.0)

#         for i, idx in enumerate(group_sorted.index):
#             used_capital = used_per_row[i]
#             result_df.at[idx, "allocation"] = used_capital
#             result_df.at[idx, "estimated_lots"] = lots_per_row[i]
#             result_df.at[idx, "used_capital"] = used_capital
#             result_df.at[idx, "unused_capital"] = 0.0

#         if leftover_capital > 0:
#             leftover_idx = lots_per_row.index(min(lots_per_row))
#             target_idx = group_sorted.index[leftover_idx]
#             result_df.at[target_idx, "allocation"] += leftover_capital
#             result_df.at[target_idx, "unused_capital"] = leftover_capital
    
#     return result_df

def build_portfolio_advanced(
    data,
    capital: float,
    security_column: str = "Sec_0",
    price_column: str = "sec_0_price", 
    param_num_column: str = "param_num",
) -> pd.DataFrame:
    """
    Построение портфеля с распределением капитала по группам инструментов и параметрам.
    """
    if capital <= 0:
        raise ValueError("Капитал должен быть положительным числом")
    
    if data.empty:
        raise ValueError("DataFrame не может быть пустым")
    
    # Проверяем наличие необходимых столбцов
    required_columns = [security_column, price_column, param_num_column]
    missing_columns = [col for col in required_columns if col not in data.columns]
    if missing_columns:
        raise ValueError(f"Отсутствуют столбцы: {missing_columns}")
    
    # Создаем копию данных для работы
    result_df = data.copy()
    
    # Инициализируем новые столбцы
    result_df["allocation"] = 0.0
    result_df["estimated_lots"] = 0
    result_df["used_capital"] = 0.0
    result_df["unused_capital"] = 0.0
    
    # Получаем количество уникальных инструментов
    unique_securities = result_df[security_column].nunique()
    capital_per_security = capital / unique_securities
    
    # Группируем по инструментам
    grouped = result_df.groupby(security_column)
    
    for security_name, group in grouped:
        # Сортируем группу по номеру параметра
        group_sorted = group.sort_values(param_num_column)
        
        # Получаем цену инструмента (берем из первой строки)
        instrument_price = float(group_sorted[price_column].iloc[0])
        
        if instrument_price <= 0:
            # Если цена 0 или отрицательная, пропускаем инструмент
            continue
        
        # Количество строк в группе
        param_count = len(group_sorted)

        if param_count == 0:
            continue

        # ЖАДНЫЙ АЛГОРИТМ: выдаем лоты, работая с общим бюджетом инструмента
        remaining_capital = capital_per_security
        lots_per_row = [0] * param_count  # Количество лотов для каждой строки

        while remaining_capital >= instrument_price:
            min_lots_idx = lots_per_row.index(min(lots_per_row))
            lots_per_row[min_lots_idx] += 1
            remaining_capital -= instrument_price

        # Рассчитываем использованный капитал для каждой строки
        used_per_row = [lots * instrument_price for lots in lots_per_row]
        total_used = sum(used_per_row)
        leftover_capital = capital_per_security - total_used

        # Записываем результаты для каждой строки
        for i, idx in enumerate(group_sorted.index):
            result_df.at[idx, "estimated_lots"] = lots_per_row[i]
            result_df.at[idx, "used_capital"] = used_per_row[i]
            result_df.at[idx, "allocation"] = used_per_row[i]  # Сначала записываем только used_capital
            result_df.at[idx, "unused_capital"] = 0.0

        # Добавляем остаток к строке с минимальными лотами
        if leftover_capital > 0:
            min_lots_value = min(lots_per_row)
            leftover_idx = lots_per_row.index(min_lots_value)
            target_idx = group_sorted.index[leftover_idx]
            
            # Добавляем остаток только к allocation и unused_capital
            result_df.at[target_idx, "allocation"] += leftover_capital
            result_df.at[target_idx, "unused_capital"] = leftover_capital
    
    return result_df

In [20]:
test = build_portfolio_advanced(
    data = test_portfolio, capital = 68300)

In [21]:
test

,strategy_id,Sec_0,sec_0_price,sec_0_GO,sec_1,W_0,SM,param_num,allocation,estimated_lots,used_capital,unused_capital
0,AFZ5_1,AFZ5,5793.0,1538.73,MXZ5,0.02,1;1;0.01;0.05,1,21956.0,3,17379.0,4577.0
0,AFZ5_2,AFZ5,5793.0,1538.73,MXZ5,0.02,1;1;0.01;0.05,2,21956.0,3,17379.0,4577.0
0,AFZ5_3,AFZ5,5793.0,1538.73,MXZ5,0.02,1;1;0.01;0.05,3,21956.0,3,17379.0,4577.0


In [23]:
import pandas as pd

# Создаем точные тестовые данные как у вас
test_data = pd.DataFrame({
    'strategy_id': ['AFZ5_1', 'AFZ5_2', 'AFZ5_3'],
    'Sec_0': ['AFZ5', 'AFZ5', 'AFZ5'],
    'sec_0_price': [5793.0, 5793.0, 5793.0],
    'sec_0_GO': [1538.73, 1538.73, 1538.73],
    'sec_1': ['MXZ5', 'MXZ5', 'MXZ5'],
    'W_0': [0.02, 0.02, 0.02],
    'SM': ['1;1;0.01;0.05', '1;1;0.01;0.05', '1;1;0.01;0.05'],
    'param_num': [1, 2, 3]
})

print("Тестовые данные:")
print(test_data)

def build_portfolio_advanced_debug(
    data,
    capital: float,
    security_column: str = "Sec_0",
    price_column: str = "sec_0_price",
    param_num_column: str = "param_num",
) -> pd.DataFrame:
    """
    Отладочная версия функции с подробными выводами
    """
    print(f"\n=== ОТЛАДКА ФУНКЦИИ ===")
    print(f"Капитал: {capital}")
    print(f"Столбец инструментов: {security_column}")
    print(f"Столбец цен: {price_column}")
    print(f"Столбец параметров: {param_num_column}")
    
    if capital <= 0:
        raise ValueError("Капитал должен быть положительным числом")
    
    if data.empty:
        raise ValueError("DataFrame не может быть пустым")
    
    # Проверяем наличие необходимых столбцов
    required_columns = [security_column, price_column, param_num_column]
    missing_columns = [col for col in required_columns if col not in data.columns]
    if missing_columns:
        raise ValueError(f"Отсутствуют столбцы: {missing_columns}")
    
    # Создаем копию данных для работы
    result_df = data.copy()
    
    # Инициализируем новые столбцы
    result_df["allocation"] = 0.0
    result_df["estimated_lots"] = 0
    result_df["used_capital"] = 0.0
    result_df["unused_capital"] = 0.0
    
    # Получаем количество уникальных инструментов
    unique_securities = result_df[security_column].nunique()
    capital_per_security = capital / unique_securities
    
    print(f"\nУникальных инструментов: {unique_securities}")
    print(f"Капитал на инструмент: {capital_per_security}")
    
    # Группируем по инструментам
    grouped = result_df.groupby(security_column)
    
    for security_name, group in grouped:
        print(f"\n--- Обрабатываем инструмент: {security_name} ---")
        
        # Сортируем группу по номеру параметра
        group_sorted = group.sort_values(param_num_column)
        print(f"Группа после сортировки по {param_num_column}:")
        print(group_sorted[[param_num_column, price_column]])
        
        # Получаем цену инструмента (берем из первой строки)
        instrument_price = float(group_sorted[price_column].iloc[0])
        print(f"Цена инструмента: {instrument_price}")
        
        if instrument_price <= 0:
            print("Цена <= 0, пропускаем инструмент")
            continue
        
        # Количество строк в группе
        param_count = len(group_sorted)
        print(f"Количество параметров: {param_count}")

        if param_count == 0:
            continue

        # ЖАДНЫЙ АЛГОРИТМ
        remaining_capital = capital_per_security
        lots_per_row = [0] * param_count
        
        print(f"\nЖАДНЫЙ АЛГОРИТМ:")
        print(f"Начальный капитал: {remaining_capital}")
        print(f"Цена за лот: {instrument_price}")
        print(f"Максимум лотов: {int(remaining_capital // instrument_price)}")
        
        step = 0
        while remaining_capital >= instrument_price:
            step += 1
            min_lots_value = min(lots_per_row)
            min_lots_idx = lots_per_row.index(min_lots_value)
            lots_per_row[min_lots_idx] += 1
            remaining_capital -= instrument_price
            
            print(f"Шаг {step}: Даем лот параметру {min_lots_idx+1} (param_num={group_sorted.iloc[min_lots_idx][param_num_column]})")
            print(f"  Лоты по строкам: {lots_per_row}")
            print(f"  Остаток капитала: {remaining_capital:.2f}")

        print(f"\nИтого лотов по строкам: {lots_per_row}")
        print(f"Общее количество лотов: {sum(lots_per_row)}")
        
        # Рассчитываем использованный капитал для каждой строки
        used_per_row = [lots * instrument_price for lots in lots_per_row]
        total_used = sum(used_per_row)
        leftover_capital = capital_per_security - total_used
        
        print(f"Использованный капитал по строкам: {used_per_row}")
        print(f"Общий использованный капитал: {total_used}")
        print(f"Остаток капитала: {leftover_capital}")

        # Записываем результаты для каждой строки
        print(f"\nЗАПИСЬ РЕЗУЛЬТАТОВ:")
        for i, idx in enumerate(group_sorted.index):
            param_num_val = group_sorted.at[idx, param_num_column]
            print(f"Строка {i} (index={idx}, param_num={param_num_val}): {lots_per_row[i]} лотов, {used_per_row[i]:.0f} руб.")
            
            result_df.at[idx, "estimated_lots"] = lots_per_row[i]
            result_df.at[idx, "used_capital"] = used_per_row[i]
            result_df.at[idx, "allocation"] = used_per_row[i]
            result_df.at[idx, "unused_capital"] = 0.0

        # Добавляем остаток к строке с минимальными лотами
        if leftover_capital > 0:
            min_lots_value = min(lots_per_row)
            leftover_idx = lots_per_row.index(min_lots_value)
            target_idx = group_sorted.index[leftover_idx]
            target_param = group_sorted.at[target_idx, param_num_column]
            
            print(f"\nОСТАТОК КАПИТАЛА:")
            print(f"Добавляем остаток {leftover_capital:.2f} к строке с param_num={target_param} (index={target_idx})")
            
            # Добавляем остаток только к allocation и unused_capital
            result_df.at[target_idx, "allocation"] += leftover_capital
            result_df.at[target_idx, "unused_capital"] = leftover_capital
    
    return result_df

# Тестируем с капиталом 68300 (как в вашем примере)
capital = 68300
result = build_portfolio_advanced_debug(test_data, capital)

print(f"\n=== ФИНАЛЬНЫЙ РЕЗУЛЬТАТ ===")
print(result[['strategy_id', 'param_num', 'estimated_lots', 'used_capital', 'allocation', 'unused_capital']])

# Проверяем суммы
total_lots = result['estimated_lots'].sum()
total_used = result['used_capital'].sum()
expected_lots = int(capital // 5793.0)

print(f"\n=== ПРОВЕРКА ===")
print(f"Общее количество лотов: {total_lots}")
print(f"Ожидаемое количество лотов: {expected_lots}")
print(f"Совпадает: {'✅' if total_lots == expected_lots else '❌'}")
print(f"Общий использованный капитал: {total_used:.0f}")
print(f"Ожидаемый капитал: {expected_lots * 5793:.0f}")

Тестовые данные:
  strategy_id Sec_0  sec_0_price  sec_0_GO sec_1   W_0             SM  \
0      AFZ5_1  AFZ5       5793.0   1538.73  MXZ5  0.02  1;1;0.01;0.05   
1      AFZ5_2  AFZ5       5793.0   1538.73  MXZ5  0.02  1;1;0.01;0.05   
2      AFZ5_3  AFZ5       5793.0   1538.73  MXZ5  0.02  1;1;0.01;0.05   

   param_num  
0          1  
1          2  
2          3  

=== ОТЛАДКА ФУНКЦИИ ===
Капитал: 68300
Столбец инструментов: Sec_0
Столбец цен: sec_0_price
Столбец параметров: param_num

Уникальных инструментов: 1
Капитал на инструмент: 68300.0

--- Обрабатываем инструмент: AFZ5 ---
Группа после сортировки по param_num:
   param_num  sec_0_price
0          1       5793.0
1          2       5793.0
2          3       5793.0
Цена инструмента: 5793.0
Количество параметров: 3

ЖАДНЫЙ АЛГОРИТМ:
Начальный капитал: 68300.0
Цена за лот: 5793.0
Максимум лотов: 11
Шаг 1: Даем лот параметру 1 (param_num=1)
  Лоты по строкам: [1, 0, 0]
  Остаток капитала: 62507.00
Шаг 2: Даем лот параметру 2 (para